# Reading Assignment 1 - Learning PyTorch with Examples

## Basics

At its core, PyTorch provides two main features:
1. An n-dimensional Tensor, similar to numpy but can run on GPUs
2. Automatic differentiation for building and training neural networks

For the remainder of the notebook, the example problem will be fitting $y = \sin(x)$ with a third order polynomial. The network will have **four** parameters, and will be trained with **gradient descent** to fit random data by minimizing the Euclidian distance between the network output and the true output

### Warm-up: numpy

Before introducing PyTorch, we implement a network using numpy.

Numpy provides an **n-dimensional** array object, and many functions for manipulating these arrays. Numpy is a generic framework for scientific computing; it does not know anything about computation graphs, or deep learning, or gradients. However, we can easily use numpy to fit a **third order polynomial** to sine function by manually implementing the forward and backward passes through the network using numpy operations

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

99 1836.607952752434
199 1224.772884890247
299 818.0543752804074
399 547.5954865743989
499 367.68155738859423
599 247.95417229684
699 168.24712127286918
799 115.16060118977751
899 79.7880682221355
999 56.20757224278491
1099 40.480226561383105
1199 29.985168437900725
1299 22.977836163828915
1399 18.296488861476927
1499 15.167158420262428
1599 13.07397744047487
1699 11.67294069213182
1799 10.734531083961873
1899 10.105534260496576
1999 9.683613838924938
Result: y = 0.01574741300600636 + 0.8320507034952958 x + -0.002716690781619585 x^2 + -0.0898184316891859 x^3


### PyTorch: Tensors

Numpy can't use GPUs to accelerate its numerical computations. For modern deep neural networks, GPUs often provide speedups of 50x or greater.

The most fundamental PyTorch concept is the **Tensor**. A PyTorch Tensor is conceptually identical to a numpy array: A Tensor is an n-dimensional array, and PyTorch provides many functions for operating on these Tensors.

Behind the scenes, Tensors can keep track of a computational graph and gradients, but they're also useful as a generic tool for scientific computing. 

In the below example, we use PyTorch tensors to fit a third order polynomial to sine function, just like numpy we still need to manually implement the forward and backward passes through the network:

In [2]:
# -*- coding: utf-8 -*-

import torch
import math


dtype = torch.float
device = torch.device("cpu")
device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 1789.744140625
199 1193.173583984375
299 796.7271118164062
399 533.1870727539062
499 357.9376525878906
599 241.3577880859375
699 163.77679443359375
799 112.1279525756836
899 77.72840118408203
999 54.80735778808594
1099 39.52712631225586
1199 29.335729598999023
1299 22.534818649291992
1399 17.993938446044922
1499 14.960291862487793
1599 12.932371139526367
1699 11.57589340209961
1799 10.667959213256836
1899 10.059793472290039
1999 9.652169227600098
Result: y = 0.015077024698257446 + 0.8323007822036743 x + -0.0026010368019342422 x^2 + -0.08985400199890137 x^3


##### Autograd Example

In the previous example, we had to manually implement both the forward and backward pass of our neural network, if you remember previously one of the two main features that PyTorch provides is **automatic differentiation**.  We can use this technique to automate the computation of the forward and backward passes in neural networks

The `autograd` package in PyTorch provides this functionality. When using autograd, the forward pass of the network will define a **computationnal graph**; nodes in the graph will be Tensors, and edges will be functions that produce output Tensors from input Tensors. Backpropogating through this graph lets you easily compute gradients.

In practice, each Tensor represents a node in a computational graph. if `x` is a Tensor that has `x.requires_grad = True` then `x.grad` is another Tensor holding the gradient of `x` with respect to some scalar value. 

In the below example, we use PyTorch Tensors and autograd to implement our fitting sine wave with third order polynomial example; now we no longer need to manually implement the backward pass through the network.

In [3]:
import torch
import math

# We want to be able to train our model on an `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

dtype = torch.float
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")
torch.set_default_device(device)

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-1, 1, 2000, dtype=dtype)
y = torch.exp(x) # A Taylor expansion would be 1 + x + (1/2) x**2 + (1/3!) x**3 + ...

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

initial_loss = 1.
learning_rate = 1e-5
for t in range(5000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()

    # Calculare initial loss, so we can report loss relative to it
    if t==0:
        initial_loss=loss.item()

    if t % 100 == 99:
        print(f'Iteration t = {t:4d}  loss(t)/loss(0) = {round(loss.item()/initial_loss, 6):10.6f}  a = {a.item():10.6f}  b = {b.item():10.6f}  c = {c.item():10.6f}  d = {d.item():10.6f}')

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

Using cuda device
Iteration t =   99  loss(t)/loss(0) =   0.064685  a =   0.481626  b =   0.341175  c =   1.908371  d =   1.397403
Iteration t =  199  loss(t)/loss(0) =   0.036927  a =   0.633034  b =   0.319266  c =   1.538435  d =   1.278287
Iteration t =  299  loss(t)/loss(0) =   0.022535  a =   0.732184  b =   0.352922  c =   1.265379  d =   1.201064
Iteration t =  399  loss(t)/loss(0) =   0.014562  a =   0.804190  b =   0.392857  c =   1.066728  d =   1.134147
Iteration t =  499  loss(t)/loss(0) =   0.010015  a =   0.856563  b =   0.431539  c =   0.922237  d =   1.072366
Iteration t =  599  loss(t)/loss(0) =   0.007323  a =   0.894657  b =   0.467938  c =   0.817140  d =   1.014689
Iteration t =  699  loss(t)/loss(0) =   0.005646  a =   0.922366  b =   0.502027  c =   0.740697  d =   0.960743
Iteration t =  799  loss(t)/loss(0) =   0.004539  a =   0.942519  b =   0.533928  c =   0.685095  d =   0.910270
Iteration t =  899  loss(t)/loss(0) =   0.003760  a =   0.957179  b =   0.5637

### Defining New Autograd Functions

Under the hood, each primitive autograd operator is really two functions that operate on Tensors. The **forward** function computes Output Tensors from input Tensors. The **backward** function receives the gradient of the output Tensors with respect to some scalar value, and computes the gradient of the input Tensor with respect to that same scalar value

Inn PyTorch, we can easily define our own autograd operator by defining a subclass of `torch.autograd.Function` and implementing the `forward` and `backward` functions. We can then use our new autograd operator by constructing an instance and calling it like a function, passing Tensors containing impout data. 

In the below example, we define our model as $y = a + bP_3(c+dx)$ instead of $y = a + bx + cx^2 + dx^3$, where $P_{3}(x) = \frac{1}{2}(5x^{3} - 3x) $ is the **Legrende polynomial** of degree three. We write our own custom autograd function for computing forward and backward of $P_3$ , and use it to implment our model

In [7]:
import torch
import math


class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache tensors for
        use in the backward pass using the ``ctx.save_for_backward`` method. Other
        objects can be stored directly as attributes on the ctx object, such as
        ``ctx.my_object = my_object``. Check out `Extending torch.autograd <https://docs.pytorch.org/docs/stable/notes/extending.html#extending-torch-autograd>`_
        for further details.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

99 209.95834350585938
199 144.66018676757812
299 100.70249938964844
399 71.03519439697266
499 50.97850799560547
599 37.403133392333984
699 28.206867218017578
799 21.97318458557129
899 17.745729446411133
999 14.877889633178711
1099 12.93176555633545
1199 11.610918045043945
1299 10.714258193969727
1399 10.10548210144043
1499 9.692106246948242
1599 9.411375999450684
1699 9.220745086669922
1799 9.091286659240723
1899 9.003362655639648
1999 8.943641662597656
Result: y = -2.9753338681715036e-10 + -2.208526849746704 * P3(-1.1693186696692948e-10 + 0.2554861009120941 x)


#### Torch `nn` module

Computational graphs and autograd are very powerful for defining complex operators and automatically taking derivitaves, but they are too low level for large neural networks

When building neural networks, we frequently think of arranging the computation into **layers**, some of which have **learnable parameters** which will be *optimized* during learning 

In TensorFlow, there are packages like Keras, TensorFlow-Slim, and TFLearn that provide higher-l;evel abstractions over raw computational graphs that are useful for building neuarl networks.

In PyTorch, the `nn` package serves this same purpsoe. The `nn` package defines a set of **modules**, which are roughly equivalent to neural network layers. A module receives inut Tensors and computes output Tensors, but may also hold internal state such as Tensors containing learnable parameters. The `nn` package also defines a set of useful loss functions that are commonly used when training neural networks.

In the below example, we use the `nn` package to implement our polynomial model network

In [4]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3) 

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 1152.790771484375
199 768.3804931640625
299 513.2797241210938
399 343.9508056640625
499 231.52664184570312
599 156.86380004882812
699 107.26506042480469
799 74.3064956665039
899 52.398292541503906
999 37.830894470214844
1099 28.140884399414062
1199 21.692899703979492
1299 17.40057373046875
1399 14.54199504852295
1499 12.637426376342773
1599 11.367881774902344
1699 10.521221160888672
1799 9.956278800964355
1899 9.579107284545898
1999 9.327160835266113
Result: y = -0.010258960537612438 + 0.8369127511978149 x + 0.0017698421142995358 x^2 + -0.09051001071929932 x^3


#### Torch -` Optim package`

Up to this point, we've been updating the weights of our models by manually mutating the Tensors holding learnable parameters with `torch.no_grad()`. This is not a huge burden for simple optimization algorithms like stochastic gradient descent (SGD), but in practice we often train neural networks using more sophisticated optimizers like `ADaGrad`. `RMSProp`, `Adam`, and other. 

 The Optim is a package that "abstracts" (wraps up the general concept into a standard, easy-to-use interface). Basically provides implementations of commonly used optimization algorithms 

In the below example, we use the `nn` package to define our model as before, but we will optimize the model using the `RMSProp` algorithm provided by the `optim` package

In [5]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use RMSprop; the optim package contains many other
# optimization algorithms. The first argument to the RMSprop constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 26687.189453125
199 11886.779296875
299 4767.5078125
399 1842.0662841796875
499 1023.1531372070312
599 842.5428466796875
699 719.0897827148438
799 586.1036376953125
899 457.3326416015625
999 343.845458984375
1099 248.95248413085938
1199 172.00161743164062
1299 111.95281982421875
1399 67.72103881835938
1499 37.801841735839844
1599 20.142004013061523
1699 11.8657808303833
1799 9.258417129516602
1899 8.8594331741333
1999 8.856649398803711
Result: y = -2.0132058242694484e-09 + 0.8564565777778625 x + -2.751416650070837e-09 x^2 + -0.09290299564599991 x^3


#### Custom `nn` Modules

Sometimes you will want to specify models that are more complex than a seqience of existing Modules; for these cases we can define our own Modules by subclassing `nn.Module` and defining a `forward` which receives input Tensors and produces output Tensors using other modules or other autograd operations on Tensors.

In the below example, we implement our third order polynomial as  acustom Module subclass:

In [6]:
# -*- coding: utf-8 -*-
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters (defined 
# with torch.nn.Parameter) which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 815.1873779296875
199 552.2457885742188
299 375.38946533203125
399 256.3294982910156
499 176.10528564453125
599 121.99856567382812
699 85.47154998779297
799 60.78799819946289
899 44.0908203125
999 32.78437805175781
1099 25.120208740234375
1199 19.919307708740234
1299 16.386146545410156
1399 13.983278274536133
1499 12.347288131713867
1599 11.232149124145508
1699 10.471158981323242
1799 9.951272964477539
1899 9.595684051513672
1999 9.352197647094727
Result: y = 0.018593505024909973 + 0.8421121835708618 x + -0.003207689616829157 x^2 + -0.09124958515167236 x^3
